In [1]:
import json
import requests
from pprint import pprint
import numpy as np
import pandas as pd
import re
from sklearn.metrics import classification_report,jaccard_similarity_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report,jaccard_similarity_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.naive_bayes import MultinomialNB

## Inladen data

In [2]:
d = {"gemeente": [], "type":[], "tekst": [], "id" :[]}
df = pd.DataFrame(data=d)
df

,gemeente,id,tekst,type


In [3]:
gemeentelijst = ['aalsmeer','bodegravenreeuwijk',  'alkmaar', 'almelo', 'almere', 'amersfoort', 'amstelveen', 'amsterdam', 'arnhem', 'baarn', 'barneveld', 'beemster', 'binnenmaas', 'borne', 'boxmeer', 'buren', 'castricum', 'culemborg', 'deventer', 'dewolden', 'diemen', 'doetinchem', 'dongen', 'drimmelen', 'edamvolendam', 'ede', 'emmen', 'enschede', 'epe', 'ettenleur', 'goirle', 'gouda', 'groningen', 'hardenberg', 'heemskerk', 'heerde', 'hendrikidoambacht', 'hilvarenbeek', 'hilversum', 'hollandskroon', 'hulst', 'katwijk', 'krimpenerwaard', 'landsmeer', 'leeuwarden', 'leiden', 'leiderdorp', 'leusden', 'lingewaard', 'loonopzand', 'losser', 'maassluis', 'maastricht', 'medemblik', 'meierijstad', 'meppel', 'moerdijk', 'molenwaard', 'nieuwegein', 'nieuwkoop', 'nijkerk', 'noordoostpolder', 'noordwijk', 'noordwijkerhout', 'oisterwijk', 'oldambt', 'ommen', 'oosterhout', 'oostgelre', 'oss', 'overbetuwe', 'raalte', 'rheden', 'rhenen', 'rijswijk', 'roermond', 'roosendaal', 'rucphen', 'schagen', 'schiedam', 'soest', 'staphorst', 'steenbergen', 'steenwijkerland', 'stichtsevecht', 'texel', 'teylingen', 'utrecht', 'veenendaal', 'velsen', 'vlaardingen', 'vlissingen', 'voorst', 'waalwijk', 'wageningen', 'zaanstad', 'zandvoort', 'zeist', 'zoetermeer', 'zwartewaterland', 'zwolle']

In [4]:
for gemeente in gemeentelijst:
    print(gemeente)
    df = pd.DataFrame(data=d)
    parameters = {"from": "0"}
    took = 0
    r = requests.get("http://api.openraadsinformatie.nl/v0/" + gemeente +"/events/search", params=parameters)
    data = r.json()
    
    total = data["meta"]["total"]
    if total >= 10000: #Can only load 10.000 documents from API with this function, which is enough as well
        total = 9999
        
    while total > took:
        r = requests.get("http://api.openraadsinformatie.nl/v0/" + gemeente +"/events/search", params=parameters)
        data = r.json()
        took += 10 
        parameters["from"] = str(took)

        for article in data["events"]:
            if "name" in article.keys() and ("description" in article.keys() or "sources" in article.keys()):
                if "description" in article.keys(): #if present: Add the description of the document
                    text = article["name"] +  " " + article["description"]
                    if len(text) > 100: #Very short documents are not taken into account
                        df = df.append(pd.DataFrame([[gemeente, article["classification"], text, 
                                                      article["id"]]], columns=["gemeente", "type", "tekst", "id"]))
                
                if "sources" in article.keys(): #if present: Add the texts within the attached files 
                    for source in article["sources"]:
                        if "description" in source.keys():
                            if type(source["description"]) is str:
                                text = source["description"]
                                if len(text) > 100: #Very short documents are not taken into account
                                    df = df.append(pd.DataFrame([[gemeente, article["classification"], text, 
                                                                  source["url"]]], columns=["gemeente", "type", "tekst", "id"]))
    
    df.to_csv(gemeente + "_perdoc.csv") #Save all documents from the gemeente

aalsmeer
bodegravenreeuwijk
alkmaar
almelo
almere
amersfoort
amstelveen
amsterdam
arnhem
baarn
barneveld
beemster
binnenmaas
borne
boxmeer
buren
castricum
culemborg
deventer
dewolden
diemen
doetinchem
dongen
drimmelen
edamvolendam
ede
emmen
enschede
epe
ettenleur
goirle
gouda
groningen
hardenberg
heemskerk
heerde
hendrikidoambacht
hilvarenbeek
hilversum
hollandskroon
hulst
katwijk
krimpenerwaard
landsmeer
leeuwarden
leiden
leiderdorp
leusden
lingewaard
loonopzand
losser
maassluis
maastricht
medemblik
meierijstad
meppel
moerdijk
molenwaard
nieuwegein
nieuwkoop
nijkerk
noordoostpolder
noordwijk
noordwijkerhout
oisterwijk
oldambt
ommen
oosterhout
oostgelre
oss
overbetuwe
raalte
rheden
rhenen
rijswijk
roermond
roosendaal
rucphen
schagen
schiedam
soest
staphorst
steenbergen
steenwijkerland
stichtsevecht
texel
teylingen
utrecht
veenendaal
velsen
vlaardingen
vlissingen
voorst
waalwijk
wageningen
zaanstad
zandvoort
zeist
zoetermeer
zwartewaterland
zwolle


## Automatische labeling

In [30]:
beleidsWoorden = ['bestuur en beleid','facilitering','bevolkingsregister','jaarrekening','jaarbegroting','jaarverslag','coalitieakkoord',
                  'straatnaamgeving','kadaster','kadastrale','stelposten','begrotingsruimte','integratieuitkeringen','decentralisatieuitkeringen','accountantscontrole','ombudsfunctie','rekenkamer','reiskosten','loonkosten', 'paspoorten', 'rijbewijzen', 'verkiezingen', 'documentaire informatievoorziening ', 'inkoop', 'ozb', 'onroerend zaakbelasting ', 'parkeerbelasting', 'hondenbelasting ', 'precariobeslasting ', 'reclamebelasting ', 'vennootschapsbelasting', 'algemene uitkering ', 'mutaties reserves', 'dividend nutsbedrijven']
veiligheidWoorden = ['veiligheid', 'bibob', 'criminaliteit','criminelen','inbraken','explosieven','radicalisering','doodschouw','buitengewoon opsporingsambtenaar','crisisbeheersing', 'brandweer', 'brandbestrijding', 'rampenbestrijding', 'halt', 'apv', 'boa', 'georganiseerde criminaliteit', 'wet wapen en munitie', 'antidiscriminatiebeleid', 'dierenbescherming ', 'politie', 'leges drank en horeca', 'openbare orde']
onderwijsWoorden = ['onderwijs', 'onderwijshuisvesting', 'school', 'scholen', 'basisschool', 'bewegingsonderwijs', 'leraren', 'vandalismebestrijding', 'schoolgebouwen', 'onderwijsbeleid', 'leerlingzaken', 'leerkrachten', 'volwasseneneducatie', 'peuterspeelzaal', 'peuterspeelzalen', 'leerlingbegeleiding', 'leerlingzorg', 'leerlingenvervoer', 'schooldeelname', 'leerplicht', 'schoolverlaten', 'basisonderwijs']
economieWoorden = ['economische ontwikkeling','economie','economische','winkeliers','ondernemers', 'toerisme', 'landbouw','veeteelt','kassen','visserij','bosbouw','fruitteelt','promotie', 'toeristen', 'beurzen', 'beurs', 'jaarmarkten', 'forensenbelasting', 'toeristenbelasting', 'vermakelijkhedenretributies', 'bedrijvenloket', 'ondernemersloket', 'startende ondernemers', 'straathandel', 'markten', 'biz-bijdrage', 'marktgelden', 'grondexploitatie bedrijventerreinen', 'winkelgebieden', 'bedrijfslocaties']
verkeerWoorden = ['verkeer', 'verkeersbeleid', 'verkeersmaatregelen', 'verkeersveiligheid', 'wegen', 'pleinen', 'fietspad', 'voetpad', 'straten', 'civieltechnische kunstwerken', 'straatverlichting', 'gladheidbestrijding', 'sneeuwruimen', 'strooien', 'straatreiniging', 'zwerfafval', 'parkeerbeleid', 'parkeermeters', 'parkeervoorzieningen ', 'jachthaven ', 'bruggelden', 'passantenhaven', 'liggelden', 'havengelden', 'baggerwerkzaamheden ', 'zeehavens', 'binnenhavens ', 'doorgaande waterwegen', 'waterkering ', 'afwatering', 'openbaar vervoer', 'bus', 'tram', 'metro', 'veerdiensten', 'taxivervoer', 'busstation', 'metrostation', 'multimodaal knooppunt']
cultuurWoorden = ['sport en cultuur', 'sportbeleid', 'topsport', 'sportbeoefening', 'recreatie', 'sport', 'cultuur', 'recreatieve', 'sportaccommodaties','sportverenigingen', 'sportvelden', 'zwembad', 'schaatshal', 'trapveldje', 'voetbalveld', 'voetbal', 'hockey', 'volleybal', 'handbal', 'muziek', 'dans', 'toneel', 'kunst', 'cultuurparticipatie', 'cultuurpresentatie', 'cultuurproductie', 'cultuuruitingen', 'kunstenaars', 'kunstwerken', 'cultuureducatie', 'herdenking', 'musea', 'museum', 'expositie', 'archeologie','archeologisch', 'heemkunde', 'historische archieven', 'bibliotheken', 'bibliotheek','stadsgezicht','monumenten','monumentenwet','lokale pers', 'lokale omroep', 'lokale informatievoorziening', 'natuurbescherming', 'onderhoud van bos', 'vijvers', 'vijver', 'openbaar groen', 'betuining', 'hobbyclub', 'volkstuinvereniging', 'speelvoorziening', 'recreatievoorziening']
gezondheidsWoorden = ['volksgezondheid', 'milieu', 'gezondheidssituatie', 'gezondheid', 'gezondheidsbevordering', 'infectieziekten', 'vaccinaties', 'vaccinatie','prenatale', 'psychosociale hulp', 'centrum jeugd en gezin', 'ziekenvervoer', 'ambulance', 'riolering', 'oppervlaktewater', 'rioolwaterzuivering', 'rioolheffing', 'grondwater', 'afvalwater', 'hemelwater', 'afval', 'vuilophaal', 'reinigingsrechten', 'vuilstort', 'afvalstoffenheffing', 'milieubeheer', 'geluidhinder', 'bodemsanering', 'rud', 'regionale uitvoeringsdiensten', 'ongediertebestrijding', 'recycling', 'begraafplaatsen', 'begraafplaats','crematoria', 'grafrechten', 'luchtkwaliteit', 'milieubeleid', 'milieukwaliteit']
sociaalWoorden = ['sociaal domein', 'burgerparticipatie', 'amw', 'algemeen maatschappelijk werk', 'burgerinitiatieven', 'eenzaamheidsbestrijding', 'collectief aanvullend vervoer', 'kinderopvang', 'noodopvang vluchtelingen', 'vreemdelingen', 'wijkteams', 'inkomensregelingen', 'ioaw', 'ioaz', 'wet inkomensvoorzieningen oudere en gedeeltelijk arbeidsongeschikte werkloze werknemers', 'wet inkomensvoorzieningen oudere en gedeeltelijk arbeidsongeschikte gewezen zelfstandigen', 'bijstandverlening', 'inkomensregeling', 'participatiewet', 'loonkostensubsidies', 'bijstandsverlening', 'bijstand', 'schuldhulpverlening', 'beschut werken', 'work first', 'proefplaatsen', 'participatieplaatsen', 'vrijwilligerswerk', 'detacheringsbanen', 'erkenning van verworven competenties', 'evc', 'loonkostensubsidie', 'jobcoach', 'werkplekaanpassingen', 'werkvoorzieningen', 'doventolk', 'inkomensvrijlating', 'stimuleringspremies', 'onkostenvergoedingen', 'no-riskpolis', 'inburgering', 'cursus nederlands', 'loonwaardebepaling', 'besluit bijstandsverlening zelfstandigen', 'bbz', 'maatwerkdienstverlening', 'maatwerkvoorzieningen', 'wmo', 'woningaanpassingen', 'gehandicaptenparkeerkaart ', 'eigen bijdrage', 'mantelzorg', 'pgb', 'jeugd-ggz', 'jeugdzorg', 'gescaleerde zorg', 'beschermdwonenvoorzieningen', 'opvangvoorzieningen', 'vrouwenopvang', 'veilig thuis', 'aanpak huiselijk geweld', 'beschermd wonen', 'kinderbeschermingsmaatregelen', 'jeugdreclassering ', 'ouderbijdragen']
wonenWoorden = ['volkshuisvesting en ruimtelijke ordening', 'ruimtelijke ordening', 'bgt', 'basisregistratie grootschalige topografie', 'bestemmingsplan', 'bestemmingsplannen', 'grondbeleid', 'cai', 'breedband', 'glasvezel', 'grondexploitatie', 'grondverwerving', 'bouwrijpe gronden', 'bouwgrondcomplexen', 'gebiedsontwikkeling', 'woningvoorraad', 'huisvestingsvoorziening', 'basisregistratie adressen en gebouwen', 'bouwtoezicht', 'bag', 'woningbouw', 'woningverbetering', 'woonruimteverdeling', 'woningsplitsingsvergunning', 'woonvergunning', 'stedelijke vernieuwing', 'woningvoorraad', 'woonomgeving']
lijsten = [beleidsWoorden,veiligheidWoorden,onderwijsWoorden,economieWoorden,verkeerWoorden,sociaalWoorden,wonenWoorden,gezondheidsWoorden,cultuurWoorden]

def preprocess(text):
    text = text.lower()
    text = re.sub("<\w*>", '', text)
    text = re.sub("<\w*\s\/>", '', text)
    text = re.sub("^https?:\/\/.*[\r\n]*", '', text)
    text = re.sub('\\\\x\S.', '', text)
    text = re.sub('[^a-z\s]', '', text)
    text = re.sub("\s+", " ", text)
    return text

def dropDuplicates(df):
    df["tekst"] = df["tekst"].apply(preprocess)
    df = df.drop_duplicates(["id"]) #Verwijder dubbele berichten
    df = df.drop_duplicates(["tekst"]).drop("Unnamed: 0", axis=1).reset_index() #Verwijder dubbele berichten
    df["sent_length"] = df['tekst'].str.split().apply(len)
    return df.loc[(df['sent_length'] > 100)]

def labeling(df, lijsten, gemeente):
    #df["sent_length"] = df['tekst'].str.split().apply(len)
    df = dropDuplicates(df)
    for lijst in lijsten:
        regex = r"\b(" 
        for word in lijst:
            regex += word + "|" 
        regex = regex[:-1]
        regex += r")\b"
        df[lijst[0]] = 0
        df[lijst[0]] = df['tekst'].str.count(regex) /df["sent_length"]
    df = df.loc[(df['sent_length'] > 100) & ((df['volksgezondheid'] > 0.01) | (df['veiligheid'] > 0.01) | (df['bestuur en beleid'] > 0.01) | (df['onderwijs'] > 0.01)| (df['economische ontwikkeling'] > 0.01)| (df['verkeer'] > 0.01)| (df['sociaal domein'] > 0.01)| (df['volkshuisvesting en ruimtelijke ordening'] > 0.01)| (df['sport en cultuur'] > 0.01)) ]
    df.to_csv(gemeente + "_perdoc_labeled.csv")
    return

In [31]:
gemeentelijst = ['aalsmeer','bodegravenreeuwijk',  'alkmaar', 'almelo', 'almere', 'amersfoort', 'amstelveen', 'amsterdam', 'arnhem', 'baarn', 'barneveld', 'beemster', 'binnenmaas', 'borne', 'boxmeer', 'buren', 'castricum', 'culemborg', 'deventer', 'dewolden', 'diemen', 'doetinchem', 'dongen', 'drimmelen', 'edamvolendam', 'ede', 'emmen', 'enschede', 'epe', 'ettenleur', 'goirle', 'gouda', 'groningen', 'hardenberg', 'heemskerk', 'heerde', 'hendrikidoambacht', 'hilvarenbeek', 'hilversum', 'hollandskroon', 'hulst', 'katwijk', 'krimpenerwaard', 'landsmeer', 'leeuwarden', 'leiden', 'leiderdorp', 'leusden', 'lingewaard', 'loonopzand', 'losser', 'maassluis', 'maastricht', 'medemblik', 'meierijstad', 'meppel', 'moerdijk', 'molenwaard', 'nieuwegein', 'nieuwkoop', 'nijkerk', 'noordoostpolder', 'noordwijk', 'noordwijkerhout', 'oisterwijk', 'oldambt', 'ommen', 'oosterhout', 'oostgelre', 'oss', 'overbetuwe', 'raalte', 'rheden', 'rhenen', 'rijswijk', 'roermond', 'roosendaal', 'rucphen', 'schagen', 'schiedam', 'soest', 'staphorst', 'steenbergen', 'steenwijkerland', 'stichtsevecht', 'texel', 'teylingen', 'utrecht', 'veenendaal', 'velsen', 'vlaardingen', 'vlissingen', 'voorst', 'waalwijk', 'wageningen', 'zaanstad', 'zandvoort', 'zeist', 'zoetermeer', 'zwartewaterland', 'zwolle']
d = {"gemeente": [], "type":[], "tekst": [], "id" :[]}
df = pd.DataFrame(data=d)
for gemeente in gemeentelijst:
    labeling(pd.read_csv(gemeente + "_perdoc.csv"), lijsten, gemeente)

In [32]:
tel = {"totaal":0, "bestuur en beleid":0,"economische ontwikkeling":0,"veiligheid":0,"sociaal domein":0,"volksgezondheid":0,"volkshuisvesting en ruimtelijke ordening":0,"onderwijs":0,"verkeer":0, "sport en cultuur":0}


for gemeente in gemeentelijst:
    df = pd.read_csv(gemeente + "_perdoc_labeled.csv").reset_index()
    tel["totaal"] += len(df)
    tel["bestuur en beleid"] += len(df.loc[df['bestuur en beleid'] > 0.01])
    tel["economische ontwikkeling"] += len(df.loc[df['economische ontwikkeling'] > 0.01])
    tel["veiligheid"] += len(df.loc[df["veiligheid"] > 0.01])
    tel["sociaal domein"] += len(df.loc[df['sociaal domein'] > 0.01])
    tel["volksgezondheid"] += len(df.loc[df['volksgezondheid'] > 0.01])
    tel["volkshuisvesting en ruimtelijke ordening"] += len(df.loc[df['volkshuisvesting en ruimtelijke ordening'] > 0.01])
    tel["onderwijs"] += len(df.loc[df['onderwijs'] > 0.01])
    tel["verkeer"] += len(df.loc[df['verkeer'] > 0.01])
    tel["sport en cultuur"] += len(df.loc[df['sport en cultuur'] > 0.01])

pprint(tel)

{'bestuur en beleid': 9788,
 'economische ontwikkeling': 5839,
 'onderwijs': 9520,
 'sociaal domein': 19082,
 'sport en cultuur': 10453,
 'totaal': 91663,
 'veiligheid': 6798,
 'verkeer': 6092,
 'volksgezondheid': 7850,
 'volkshuisvesting en ruimtelijke ordening': 23838}


## TF-IDF

In [33]:
d = {"gemeente": [], "type":[], "tekst": [], "id" :[]}
df = pd.DataFrame(data=d)
for gemeente in gemeentelijst:
    df = df.append(pd.read_csv(gemeente + "_perdoc_labeled.csv"))
    
df = df.reset_index().drop("Unnamed: 0", axis=1).drop("level_0", axis=1).drop("index", axis=1)

In [29]:
i = 0
trydr = df[df["tekst"].str.slice(0,25).str.contains("agenda")]
for row in trydr[trydr["tekst"].str.slice(0,30).str.contains("agendapunt|raadsvoorstel")==False]["id"]:
    print(row)
    i+=1
print(i)
print(len(df))

https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=3e9514a4-0e2b-4170-89b4-edfd563f2530
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=8ae9a741-536e-427d-bbe6-41a5ea4fad0b
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=1e81c468-670c-45fd-a6a7-c2bb8294bdd1
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=0e8aafe4-3685-4d9e-bfa9-713c3a061711
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=2a663c27-2fde-4dfa-9df2-99a62aaa0236
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=293790a8-5a02-45e5-ab83-67303de9073c
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=97d50ea8-6351-4a72-90cf-865f22a56116
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=d3d0b156-f601-4776-8b8c-cbd56e0ad29b
https://online.ibabs.eu/ibabsapi/publicdownload.aspx?site=aalsmeer&id=85bc5c92-a585-4081-bce0-6539e672ba04
https://online.ibabs.eu/ibabsapi/publ

In [34]:
stop = []
for lijst in lijsten:
    for word in lijst:
        stop.extend(word.split())

#df["tekst"].apply(lambda x: [item for item in x if item not in stop])
df = df.sample(frac=1).reset_index(drop=True)
print("shuffle done")
v = TfidfVectorizer(max_df=0.5,min_df=0.001, stop_words=stop)
print("TF trained")
x = v.fit_transform(df["tekst"])
print("TF executed")

shuffle done
TF trained
TF executed


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
for tekst, idc, gemeente in zip(df["tekst"][:100],df["id"][:100],df["gemeente"][:100]):
    r = requests.get("http://api.openraadsinformatie.nl/v0/"+gemeente+"/events/" + idc)
    data = r.json()
    print(idc, tekst)

In [ ]:
x

In [35]:
# minder mooie maar korte oplossing
clfs = []
for lijst in lijsten:
    #print(df[lijst[0])
    y = df[lijst[0]]
    y= y.apply(lambda x: round(x+0.493))
    
    clf = LogisticRegression(C= 10, penalty= 'l2', solver= 'liblinear',dual=False,max_iter=250).fit(x[:-15000], y[:-15000])
    prediction = clf.predict_proba(x[-15000:])[:,0]
    for threshold in [.3,.4,.5,.6,.7,.8,.9,.95]:
        print(threshold, f1_score(y[-15000:], prediction<threshold))
    print()
    
    #print(lijst[0], f1_score(y[-5000:], clf.predict(x[-5000:])))
    #print(precision_score(y[-5000:], clf.predict(x[-5000:])) , recall_score(y[-5000:], clf.predict(x[-5000:])))
    #print(clf.predict(x[-5:]))
    #print(v.get_feature_names()[np.where(clf.coef_[0] == max(clf.coef_[0]))[0][0]])
    clfs.append(clf)
    #print(sum(y[-15000:]))

0.3 0.7325660121868652
0.4 0.7645358175393511
0.5 0.78451691557452
0.6 0.7967573827446439
0.7 0.7847071583514099
0.8 0.759493670886076
0.9 0.6877520696242835
0.95 0.6021466905187836

0.3 0.8123861566484517
0.4 0.8321554770318021
0.5 0.8446937014667818
0.6 0.8524727577535625
0.7 0.853225806451613
0.8 0.8375286041189931
0.9 0.7691284098469726
0.95 0.6713208600949455

0.3 0.8222585924713585
0.4 0.8418043202033036
0.5 0.8570550261297264
0.6 0.8621929298981426
0.7 0.8632552760913559
0.8 0.8447712418300654
0.9 0.7870036101083032
0.95 0.6915926461474903

0.3 0.7484094852515905
0.4 0.7748049052396878
0.5 0.7944621938232161
0.6 0.804691483936767
0.7 0.8026565464895636
0.8 0.7788461538461539
0.9 0.6996699669966998
0.95 0.5899880810488678

0.3 0.7255113024757804
0.4 0.7628336755646818
0.5 0.7939600584510472
0.6 0.8026377767310411
0.7 0.8050201703272076
0.8 0.7796751353602666
0.9 0.710951526032316
0.95 0.6140035906642728

0.3 0.8765743073047859
0.4 0.888888888888889
0.5 0.8942077549066539
0.6 0.89

In [36]:
import pickle
for i,clf in enumerate(clfs):
    pickle.dump(clf, open("Uiteindelijke versie/Models/"+lijsten[i][0]+".pickle", "wb"))
pickle.dump(v, open("Uiteindelijke versie/Models/tfidf.pickle", "wb"))

In [26]:
from sys import getsizeof

for name in dir():
    myvalue = eval(name)
    print(name, "is", type(name), "and has size ", getsizeof(myvalue))

CountVectorizer is <class 'str'> and has size  1056
In is <class 'str'> and has size  344
LogisticRegression is <class 'str'> and has size  1464
MultinomialNB is <class 'str'> and has size  1056
OneVsOneClassifier is <class 'str'> and has size  1056
OneVsRestClassifier is <class 'str'> and has size  1056
Out is <class 'str'> and has size  368
SGDClassifier is <class 'str'> and has size  1056
TfidfVectorizer is <class 'str'> and has size  1464
_ is <class 'str'> and has size  1461138900
_18 is <class 'str'> and has size  56
_2 is <class 'str'> and has size  100
_20 is <class 'str'> and has size  1465423764
_21 is <class 'str'> and has size  10012334
_23 is <class 'str'> and has size  1461138900
_8 is <class 'str'> and has size  100
__ is <class 'str'> and has size  10012334
___ is <class 'str'> and has size  1465423764
__builtin__ is <class 'str'> and has size  80
__builtins__ is <class 'str'> and has size  80
__doc__ is <class 'str'> and has size  113
__loader__ is <class 'str'> and ha

In [ ]:
df = df.drop(["gemeente", "type", "tekst"],axis=1)

In [ ]:
# mooie maar lange oplossing
clfs = []
for lijst in lijsten:
    #print(df[lijst[0])
    idx = df.index[(df[lijst[0]] > 0.01) | (df[lijst[0]] < 0.001)]
    y = df[lijst[0]]
    y = y.loc[idx]
    y= y.apply(lambda x: round(x+0.493))
    clf = LogisticRegression(C= 10, penalty= 'l2', solver= 'liblinear',dual=False,max_iter=250).fit(x[idx[:-5000]], y[:-5000])
    prediction = clf.predict_proba(x[idx[-5000:]])[:,0]
    for threshold in [.3,.4,.5,.6,.7,.8,.9,.95]:
        print(threshold, f1_score(y[-5000:], prediction<threshold))
    clfs.append(clf)
    print(sum(y[-5000:]))

In [18]:
del x
del newx
del y


NameError: name 'newx' is not defined

In [12]:
gemeente = "teylingen"

In [13]:
df_zwolle = pd.read_csv(gemeente + "_perdoc.csv")
df_zwolle_label = pd.read_csv(gemeente + "_perdoc_labeled.csv")
df_zwolle = dropDuplicates(df_zwolle)
df_zwolle = df_zwolle[~df_zwolle["id"].isin(df_zwolle_label["id"])].dropna()
x_zwolle = v.transform(df_zwolle["tekst"])


In [14]:
print(len(clf.predict(x_zwolle)))
print(len(df_zwolle))

4463
4463


In [15]:
for clf, lijst in zip(clfs, lijsten):
    df_zwolle[lijst[0]] = clf.predict_proba(x_zwolle)[:,0]
    regex = r"\b(" 
    for word in lijst:
        regex += word + "|" 
    regex = regex[:-1]
    regex += r")\b"
    extra = (df_zwolle['tekst'].str.count(regex) /df_zwolle["sent_length"]) * 20
    df_zwolle[lijst[0]] -= extra
    #df_zwolle[lijst[0]] = (df_zwolle[lijst[0]] < 0.7)*1

In [16]:
print(sum(df_zwolle["volksgezondheid"])+sum(df_zwolle["bestuur en beleid"])+sum(df_zwolle["veiligheid"])+sum(df_zwolle["onderwijs"])+
     sum(df_zwolle["economische ontwikkeling"])+sum(df_zwolle["verkeer"])+sum(df_zwolle["sociaal domein"])+sum(df_zwolle["sport en cultuur"])
     +sum(df_zwolle["volkshuisvesting en ruimtelijke ordening"]))
print(len(df_zwolle))
print(gemeente)

35458.86510146512
4463
teylingen


In [36]:
print(df_zwolle[df_zwolle["bestuur en beleid"] < 0.7][-10:-5])
for row in df_zwolle[df_zwolle["bestuur en beleid"] < 0.7]["id"][-10:-5]:
    print(row)

      index   gemeente                                                 id  \
7209  10586  teylingen  https://api1.ibabs.eu/publicdownload.aspx?site...   
7220  10597  teylingen  https://api1.ibabs.eu/publicdownload.aspx?site...   
7275  10652  teylingen  https://api1.ibabs.eu/publicdownload.aspx?site...   
7308  10691  teylingen  https://api1.ibabs.eu/publicdownload.aspx?site...   
7313  10696  teylingen  https://api1.ibabs.eu/publicdownload.aspx?site...   

                                                  tekst        type  \
7209  raadsvoorstel besluitvormend bouwverordening t...  Agendapunt   
7220  geachte nieuwe leden van het algemeen bestuur ...  Agendapunt   
7275  aan de gemeenteraden de colleges van burgemees...  Agendapunt   
7308  brief aan de leden tav het college en de raad ...  Agendapunt   
7313  bijlage bij logabrief tazu voorbeeld eric werk...  Agendapunt   

      sent_length  bestuur en beleid  veiligheid  onderwijs  \
7209         1069           0.569536    0.99942

In [20]:
r = requests.get("http://api.openraadsinformatie.nl/v0/teylingen/events/6b4a4f2a10e203b61caf515cf43967951e93c342")
data = r.json()
print(data)
#print(data["sources"])

{'classification': 'Agendapunt', 'description': '[Besluitvormend] Met de voortgangsrapportages wordt de raad door het college geinformeerd over de realisatie van de programmabegroting over het lopende boekjaar waarmee invulling wordt gegeven aan het budgetrecht en de kaderstellende en controlerende rol.\r\n\r\nVoorstel is de 1e Voortgangsrapportage 2018 vast te stellen, evenals de bedragen per programma en taakveld. Inclusief dekking van het nadelige resultaat ten laste van de Algemene reserve.\r\n\r\nUpdate 7 juni 2018. \r\nAan de stukken is de memo extra krediet kozijnen voortgangs project cluster Hoogh Teylingen toegevoegd.  Het college stelt voor om aan het raadsbesluit  een 4e beslispunt toe te voegen “In te stemmen met de extra kosten ter grootte van €120.000 t.b.v. het vervangen van de kozijnen van het Clustergebouw Voorhout fase 2" en deze opdracht te verlenen. De toevoeging inclusief opdrachtverlening is afgestemd met de fractievoorzitters. ', 'end_date': '2018-06-13T20:00:00+

In [90]:
newx = v.transform([preprocess(data["sources"][0]["description"]),preprocess(data["sources"][1]["description"]),
                   preprocess(data["sources"][2]["description"]),preprocess(data["sources"][3]["description"])])
for clf in clfs:
     
    print(clf.predict_proba(newx)[:,0])

[0.99958099 0.99316127 0.99732031 0.97271375]
[0.95822879 0.9828052  0.9730074  0.96698925]
[0.97269327 0.98195911 0.96624551 0.97083931]
[0.85999169 0.99116162 0.95039377 0.98079197]
[0.01307082 0.98652308 0.98030109 0.90625074]
[0.99916742 0.98087604 0.97976985 0.91095229]
[0.99721454 0.94555974 0.95063284 0.86939343]
[0.99742911 0.99277069 0.99209843 0.99253291]
[0.99878573 0.4696413  0.72357945 0.89069046]


## Guided LDA

In [25]:
d = {"gemeente": [], "type":[], "tekst": [], "id" :[]}
df = pd.DataFrame(data=d)
for gemeente in gemeentelijst:
    print(gemeente)
    new_df = dropDuplicates(pd.read_csv(gemeente + "_perdoc.csv"))
    new_df["tekst"]
    df = df.append(new_df)

aalsmeer
bodegravenreeuwijk
alkmaar
almelo
almere
amersfoort
amstelveen
amsterdam
arnhem
baarn
barneveld
beemster
binnenmaas
borne
boxmeer
buren
castricum
culemborg
deventer
dewolden
diemen
doetinchem
dongen
drimmelen
edamvolendam
ede
emmen
enschede
epe
ettenleur
goirle
gouda
groningen
hardenberg
heemskerk
heerde
hendrikidoambacht
hilvarenbeek
hilversum
hollandskroon
hulst
katwijk
krimpenerwaard
landsmeer
leeuwarden
leiden
leiderdorp
leusden
lingewaard
loonopzand
losser
maassluis
maastricht
medemblik
meierijstad
meppel
moerdijk
molenwaard
nieuwegein
nieuwkoop
nijkerk
noordoostpolder
noordwijk
noordwijkerhout
oisterwijk
oldambt
ommen
oosterhout
oostgelre
oss
overbetuwe
raalte
rheden
rhenen
rijswijk
roermond
roosendaal
rucphen
schagen
schiedam
soest
staphorst
steenbergen
steenwijkerland
stichtsevecht
texel
teylingen
utrecht
veenendaal
velsen
vlaardingen
vlissingen
voorst
waalwijk
wageningen
zaanstad
zandvoort
zeist
zoetermeer
zwartewaterland
zwolle


In [26]:
vectorizer =CountVectorizer(min_df = 0.001, max_df = 0.5)
X = vectorizer.fit_transform(df["tekst"])

In [27]:
X

<126108x24418 sparse matrix of type '<class 'numpy.int64'>'
	with 41950715 stored elements in Compressed Sparse Row format>

In [28]:
del df

In [29]:
import guidedlda
model = guidedlda.GuidedLDA(n_topics=15, n_iter=50, random_state=7, refresh=20)
model.fit(X)

INFO:guidedlda:n_documents: 126108
INFO:guidedlda:vocab_size: 24418
INFO:guidedlda:n_words: 99703540
INFO:guidedlda:n_topics: 15
INFO:guidedlda:n_iter: 50
C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\guidedlda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:guidedlda:<0> log likelihood: -1116967334
INFO:guidedlda:<20> log likelihood: -893660275
INFO:guidedlda:<40> log likelihood: -875953411
INFO:guidedlda:<49> log likelihood: -873227146


In [31]:
vocab = vectorizer.get_feature_names()
word2id = dict((v, idx) for idx, v in enumerate(vocab))

In [32]:
topic_word = model.topic_word_
n_top_words = 10
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: artikel lid verordening bestuur per leden wethouders voorzitter regeling wet
Topic 1: bestemmingsplan ten gebied ruimtelijke binnen onder plangebied meter woningen plan
Topic 2: aantal jaar ra per veiligheid jongeren cu politie meer aanpak
Topic 3: katwijk wethouders nvt almere stemmen gemeenteraad bijlagen uitvoering advies dd
Topic 4: heer wethouder motie dhr vergadering cda voorzitter pvda vvd stemmen
Topic 5: ik wij we heer voorzitter maar wel wat daar nog
Topic 6: artikel verordening lid wet onder indien eerste aanvraag bedoeld ten
Topic 7: begroting kosten jaar reserve per financile bedrag miljoen ten algemene
Topic 8: gemeenten bestuur begroting gemeenschappelijke regeling samenwerking algemeen regio organisatie financile
Topic 9: bestemmingsplan vast zienswijzen ruimtelijke vaststelling plan zienswijze gemeenteraad dd inzage
Topic 10: we wij meer onze ons wat maar zich andere nieuwe
Topic 11: commissie uur brief agenda stukken vergadering vragen gemeenteraad dd voorzit

In [33]:
from six.moves import cPickle as pickle
with open('lda_model85.pickle', 'wb') as file_handle:
    pickle.dump(model, file_handle)
# # load the model for prediction
# with open('guidedlda_model.pickle', 'rb') as file_handle:
#     model = pickle.load(file_handle)


In [34]:
seed_topic_list = lijsten

model = guidedlda.GuidedLDA(n_topics=20, n_iter=50, random_state=7, refresh=20)

seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        if word in word2id.keys():
            seed_topics[word2id[word]] = t_id

model.fit(X, seed_topics=seed_topics, seed_confidence=0.85)

INFO:guidedlda:n_documents: 126108
INFO:guidedlda:vocab_size: 24418
INFO:guidedlda:n_words: 99703540
INFO:guidedlda:n_topics: 20
INFO:guidedlda:n_iter: 50
C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\guidedlda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:guidedlda:<0> log likelihood: -1145682961
INFO:guidedlda:<20> log likelihood: -893608230
INFO:guidedlda:<40> log likelihood: -879777519
INFO:guidedlda:<49> log likelihood: -877482119


In [35]:
Z = vectorizer.transform(["vaccinatie gezondheid milieu"])
model.transform(Z)
Z = vectorizer.transform(["sporten cultuur"])
model.transform(Z)

C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\guidedlda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


array([[4.11663438e-05, 2.18553747e-08, 1.20362364e-03, 1.75196411e-03,
        1.75932493e-08, 1.41802623e-08, 4.90238346e-05, 1.63402140e-08,
        9.92882129e-01, 8.62665985e-05, 2.73331240e-04, 3.23906260e-04,
        7.86876105e-04, 1.64899082e-08, 6.19884907e-05, 3.32514220e-05,
        4.99998626e-04, 1.66408828e-03, 9.71847609e-05, 2.45115276e-04]])

In [20]:
vocab = vectorizer.get_feature_names()
word2id = dict((v, idx) for idx, v in enumerate(vocab))

topic_word = model.topic_word_
n_top_words = 10
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: begroting bestuur gemeenteraad gemeenschappelijke regeling financile jaarrekening raadsvoorstel vast burgemeester
Topic 1: artikel lid verordening wet onder burgemeester eerste indien bedoeld vergunning
Topic 2: artikel verordening onderwijs lid school wet scholen kosten ouders voorziening
Topic 3: we meer samenwerking nieuwe regio gemeenten visie samen andere wat
Topic 4: ruimte utrecht bewoners verkeer zal maatregelen parkeren mogelijk gebied ten
Topic 5: gemeenten zorg sociaal ondersteuning wmo we werk domein sociale mensen
Topic 6: bestemmingsplan ruimtelijke plan zienswijzen vast ten woningen artikel vaststelling gebied
Topic 7: begroting per gemeenten bestuur kosten jaar verordening ggd tarief tarieven
Topic 8: reserve kosten financile jaar programma bedrag begroting beschikbaar ten nog
Topic 9: heer motie cda vvd stemmen pvda fractie mevrouw amendement voorzitter
Topic 10: ik wij we voorzitter maar heer wat wel daar nog
Topic 11: we wij wat maar meer nog vragen ons vraa

In [16]:
stop = []
for lijst in lijsten:
    for word in lijst:
        stop.extend(word.split())

#df["tekst"].apply(lambda x: [item for item in x if item not in stop])
df = df.sample(frac=1).reset_index(drop=True)
x = vectorizer.transform(df["tekst"])

NameError: name 'df' is not defined

In [ ]:
x = model.transform(x)

In [ ]:
clfs = []
for lijst in lijsten:
    y = df[lijst[0]]
    y= y.apply(lambda x: round(x+0.493))
    clf = LogisticRegression(C= 10, penalty= 'l2', solver= 'liblinear',dual=False,max_iter=250).fit(x[:-5000], y[:-5000])
    prediction = clf.predict_proba(x[-5000:])[:,0]
    for threshold in [.3,.4,.5,.6,.7,.8,.9,.95]:
        print(threshold, f1_score(y[-5000:], prediction<threshold))
    
    #print(lijst[0], f1_score(y[-5000:], clf.predict(x[-5000:])))
    #print(precision_score(y[-5000:], clf.predict(x[-5000:])) , recall_score(y[-5000:], clf.predict(x[-5000:])))
    #print(clf.predict(x[-5:]))
    #print(v.get_feature_names()[np.where(clf.coef_[0] == max(clf.coef_[0]))[0][0]])
    clfs.append(clf)
    print(sum(y[-5000:]))

In [ ]:
    lijsten = [beleidsWoorden,veiligheidWoorden,onderwijsWoorden,economieWoorden,
               verkeerWoorden,sociaalWoorden,wonenWoorden,gezondheidsWoorden,cultuurWoorden]
